In [5]:
import glob
import os
import random
import h5py
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import torch
from copy import deepcopy

output_dir = "/mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks"
all_files = sorted(glob.glob(os.path.join(output_dir, "*.h5")))

for file in random.sample(all_files, 1):
    with h5py.File(file, "r") as f:
        grp = f["results"]
        graph_properties = pickle.loads(grp["graph_properties"][()])
        # Tensors erstellen
        nodes = torch.tensor(graph_properties[0])          # Shape [199]
        edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
        coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]
        parental_nodes_hgt_events_corrected = grp.attrs["parental_nodes_hgt_events_corrected"]
        gene_absence_presence_matrix = grp.attrs["gene_absence_presence_matrix"] 

# Erstelle einen gerichteten Graphen
G = nx.DiGraph()

# Füge Knoten hinzu (optional mit Koordinaten als Attribut)
for i, node_id in enumerate(nodes.tolist()):
    G.add_node(node_id, core_distance = coords[:, i].tolist()[0], allele_distance = coords[:, i].tolist()[1])

# Füge Kanten hinzu
edge_list = edges.tolist()
for src, dst in zip(edge_list[0], edge_list[1]):
    G.add_edge(src, dst)

G = G.reverse()

# 1. Leaves identifizieren (Knoten ohne Nachfolger)
leaves = [n for n in G.nodes if G.out_degree(n) == 0]

# 2. Initialisiere Level-Dictionary
levels = {n: 0 for n in leaves}  # Leaves haben Level 0

# 3. Topologische Sortierung (damit Kinder vor Eltern behandelt werden)
for node in reversed(list(nx.topological_sort(G))):
    successors = list(G.successors(node))
    if successors:
        levels[node] = 1 + max(levels[s] for s in successors)

# 4. Level als Attribut setzen
nx.set_node_attributes(G, levels, "level")

G = G.reverse()

H = deepcopy(G)

# Berechne neuen Wert basierend auf Differenz zum Summenwert der Children
for node in G.nodes():
    children = list(G.predecessors(node))
    if children:
        core_sum = sum(G.nodes[child]['core_distance'] for child in children)
        allele_sum = sum(G.nodes[child]['allele_distance'] for child in children)
        H.nodes[node]['core_distance'] = G.nodes[node]['core_distance'] - core_sum
        H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance'] - allele_sum
    else:
        H.nodes[node]['core_distance'] = G.nodes[node]['core_distance']
        H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance']


### PLOT

from pyvis.network import Network
from collections import defaultdict
import networkx as nx

# Netzwerk initialisieren
net = Network(height="900px", width="100%", directed=True)
net.set_options("""
{
    "layout": {
      "hierarchical": {
        "enabled": true,
        "direction": "DU",
        "sortMethod": "directed",
        "levelSeparation": 50,
        "nodeSpacing": 200
      }
    },
  "nodes": {
    "shape": "dot",
    "size": 12,
    "font": { "size": 16 }
  },
  "edges": {
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    }
  }
}
""")

G = H

# Gruppiere Nodes nach Level
levels = defaultdict(list)
for n in G.nodes:
    level = G.nodes[n].get("level", 0)
    levels[level].append(int(n))

def descendants_count(node):
    return len(nx.descendants(G, node))

# Level sortieren, damit Level 0 zuerst kommt, dann aufsteigend
for level in sorted(levels.keys()):
    nodes_in_level = levels[level]
    print(nodes_in_level)
    if level == 0:
        # Nur Blätter mit n < 100 und numerisch sortieren
        leaves_level_0 = [n for n in sorted(nodes_in_level) if n < 100]
        nodes_sorted = sorted(leaves_level_0)
    #else:
        # Andere Levels: sortiere nach Anzahl Nachkommen
    #    nodes_sorted = sorted(nodes_in_level, key=descendants_count)
    nodes_sorted = nodes_in_level
        
    for n in nodes_sorted:
        core = G.nodes[n].get('core_distance', 0)
        allele = G.nodes[n].get('allele_distance', 0)
        title = f"Core: {core:.2f}, Allele: {allele:.2f}"
        label = f"{n}\n({core:.2f}, {allele:.2f})"

        color = "lightblue"
        if n in parental_nodes_hgt_events_corrected:
            color = "red"
        elif n < 100 and gene_absence_presence_matrix[n] == 1:
            color = "green"
        elif n < 100 and gene_absence_presence_matrix[n] == 0:
            color = "black"

        net.add_node(n, label=label, title=title, color=color, level=level)

# Edges hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

import webbrowser
from pathlib import Path
import subprocess

html_file = Path("/mnt/c/Users/uhewm/OneDrive/PhD/Project No.2/pangenome/graph.html")

# Ordner sicherstellen
html_file.parent.mkdir(parents=True, exist_ok=True)

# Graph speichern
net.show(str(html_file), notebook=False)

# WSL-Pfad in Windows-Pfad umwandeln
win_path = subprocess.run(["wslpath", "-w", str(html_file)], capture_output=True, text=True).stdout.strip()

# Direkt in Chrome öffnen
subprocess.run(["cmd.exe", "/C", "start", "chrome", win_path])



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[100, 101, 102, 103, 104, 105, 106, 107, 110, 109, 113, 112, 119, 117, 116, 115, 114, 132, 130, 129, 128, 127, 126, 123, 122, 120, 157, 151, 144, 142, 141, 140, 139, 136, 180, 179, 174, 167]
[108, 111, 118, 135, 134, 133, 131, 124, 121, 159, 154, 147, 145, 138, 137, 172, 165, 162]
[125, 160, 156, 153, 152, 150, 148, 146, 143, 188, 173, 168, 166, 163]
[161, 158, 155, 149, 183, 177, 176, 175, 171, 169]
[190, 184, 182, 181, 178, 170, 164]
[192, 189, 187, 186, 185]
[194, 193, 191]
[195]
[196]
[197]
[198]
/mnt/c/Users/uhewm/OneDrive/PhD/Project No.2/pangenome/graph.html
>4;1H84l=                         

/tmp/ipykernel_4896/3934682594.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1])          # Shape [2, 198]
/tmp/ipykernel_4896/3934682594.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(graph_properties[2].T)       # Shape [2, 199]


CompletedProcess(args=['cmd.exe', '/C', 'start', 'chrome', 'C:\\Users\\uhewm\\OneDrive\\PhD\\Project No.2\\pangenome\\graph.html'], returncode=0)

In [9]:
G.order()

199

In [57]:
print(graph_properties[2])
print(G.nodes)
#print(torch.tensor([1 if node in result["parental_nodes_hgt_events"] else 0 for node in graph_properties[0]]))
#print(torch.tensor([1 if node in result["parental_nodes_hgt_events_corrected"] else 0 for node in graph_properties[0]]))
print(parental_nodes_hgt_events_corrected)
print(gene_absence_presence_matrix)
print(graph_properties[1])

tensor([[  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [  0.0000,   0.0000],
        [ 

In [41]:
from pyvis.network import Network

# Falls nicht schon passiert:
net = Network(notebook=True, height="800px", width="100%")

# Voraussetzung: Du hast `parental_nodes_hgt_events_corrected` als Set oder Liste
# Und `gene_absence_presence_matrix` als dict oder Liste

for n in G.nodes:
    core = G.nodes[n].get('core_distance', 0)
    allele = G.nodes[n].get('allele_distance', 0)
    title = f"Core: {core:.2f}, Allele: {allele:.2f}"
    label = f"{n}\n({core:.2f}, {allele:.2f})"

    # Default-Farbe
    color = "lightblue"

    # Rote Knoten: parental HGT
    if n in parental_nodes_hgt_events_corrected:
        color = "red"

    # Grüne Knoten: node < 100 und gene == 1
    elif n < 100 and gene_absence_presence_matrix[n] == 1:
        color = "green"
    elif n < 100 and gene_absence_presence_matrix[n] == 0:
        color = "black"

    net.add_node(n, label=label, title=title, color=color)

# Kanten hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

# Anzeigen
net.show("graph.html")



graph.html


In [82]:
from pyvis.network import Network
from collections import defaultdict
import networkx as nx

# Erstelle einen gerichteten Graphen
G = nx.DiGraph()

# Füge Knoten hinzu (optional mit Koordinaten als Attribut)
for i, node_id in enumerate(nodes.tolist()):
    G.add_node(node_id, core_distance = coords[:, i].tolist()[0], allele_distance = coords[:, i].tolist()[1])

# Füge Kanten hinzu
edge_list = edges.tolist()
for src, dst in zip(edge_list[0], edge_list[1]):
    G.add_edge(src, dst)

G = G.reverse()

# 1. Leaves identifizieren (Knoten ohne Nachfolger)
leaves = [n for n in G.nodes if G.out_degree(n) == 0]

# 2. Initialisiere Level-Dictionary
levels = {n: 0 for n in leaves}  # Leaves haben Level 0

# 3. Topologische Sortierung (damit Kinder vor Eltern behandelt werden)
for node in reversed(list(nx.topological_sort(G))):
    successors = list(G.successors(node))
    if successors:
        levels[node] = 1 + max(levels[s] for s in successors)

# 4. Level als Attribut setzen
nx.set_node_attributes(G, levels, "level")

G = G.reverse()


# Netzwerk initialisieren
net = Network(height="900px", width="100%", directed=True)
net.set_options("""
{
  "layout": {
    "hierarchical": {
      "enabled": true,
      "direction": "DU",
      "sortMethod": "directed",
      "levelSeparation": 100,
      "nodeSpacing": 150,
      "treeSpacing": 200
    }
  },
  "nodes": {
    "shape": "dot",
    "size": 12,
    "font": { "size": 16 }
  },
  "edges": {
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    }
  }
}
""")


# Gruppiere Nodes nach Level
levels = defaultdict(list)
for n in G.nodes:
    level = G.nodes[n].get("level", 0)
    levels[level].append(int(n))

def descendants_count(node):
    return len(nx.descendants(G, node))

# Level sortieren, damit Level 0 zuerst kommt, dann aufsteigend
for level in sorted(levels.keys()):
    nodes_in_level = levels[level]

    if level == 0:
        # Nur Blätter mit n < 100 und numerisch sortieren
        leaves_level_0 = [n for n in nodes_in_level if n < 100]
        nodes_sorted = sorted(leaves_level_0)
    #else:
        # Andere Levels: sortiere nach Anzahl Nachkommen
    #    nodes_sorted = sorted(nodes_in_level, key=descendants_count)
    nodes_sorted = nodes_in_level
    
    for n in nodes_sorted:
        core = G.nodes[n].get('core_distance', 0)
        allele = G.nodes[n].get('allele_distance', 0)
        title = f"Core: {core:.2f}, Allele: {allele:.2f}"
        label = f"{n}\n({core:.2f}, {allele:.2f})"

        color = "lightblue"
        if n in parental_nodes_hgt_events_corrected:
            color = "red"
        elif n < 100 and gene_absence_presence_matrix[n] == 1:
            color = "green"
        elif n < 100 and gene_absence_presence_matrix[n] == 0:
            color = "black"

        net.add_node(n, label=label, title=title, color=color, level=level)

# Edges hinzufügen
for u, v in G.edges:
    net.add_edge(u, v)

net.show("graph.html", notebook=False)


graph.html


In [76]:
import networkx as nx
from pyvis.network import Network

# === 1. Beispielgraph ===
G = nx.Graph()
G.add_edges_from([
    (1, 2), (1, 3), (2, 4), (3, 5), (4, 6)
])

# === 2. Layout vorbereiten ===
pos = nx.spring_layout(G, seed=42)  # Fixierte Positionen

# === 3. Visualisierung mit PyVis ===
net = Network(height="600px", width="100%", notebook=False)
net.barnes_hut()  # Oder .force_atlas_2based()

# === 4. Knoten hinzufügen mit fixierten Positionen ===
for node in G.nodes:
    x, y = pos[node]
    net.add_node(
        n_id=str(node),         # Wichtig: String!
        label=str(node),
        x=x*1000, y=y*1000,     # Skalierung, da pyvis Pixel erwartet
        fixed=True,             # Position fixieren
        physics=False           # Keine automatische Bewegung
    )

# === 5. Kanten hinzufügen ===
for u, v in G.edges:
    net.add_edge(str(u), str(v))

# === 6. Anzeigen ===
net.show("graph.html")


graph.html


AttributeError: 'NoneType' object has no attribute 'render'